# Explore the Linkedin API
https://pypi.python.org/pypi/python-linkedin/4.0

In [ ]:
import os

from linkedin import linkedin
import requests

import pandas as pd
from IPython.display import display
#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets

In [ ]:
access_token = os.getenv('LINKEDIN_ACCESS_TOKEN')
application = linkedin.LinkedInApplication(token=access_token)

https://developer.linkedin.com/docs/fields/company-profile

In [ ]:
# Search URL is https://api.linkedin.com/v1/company-search:(companies:(name,universal-name,website-url))?keywords=apple%20microsoft
search = application.search_company(
    selectors=[{'companies': ['id',
                              'name',
                              'universal-name',
                              {'company-type': 'name'},
                              {'industries': 'name'},
                              {'status': 'name'},
                              {'employee-count-range': 'name'},
                              'specialties',
                              'description',
                              'locations:(address:(city))'
                             ]
    }],
    params={'keywords': 'data'}
)
response = search['companies']['values'] # List of dictionaries
response[0].keys()
[response['locations'] for response in response]
#df = pd.DataFrame.from_dict(search['companies']['values'])
#df.head()

https://developer.linkedin.com/docs/fields/basic-profile

https://developer.linkedin.com/docs/fields/positions

https://developer.linkedin.com/docs/fields/companies

In [ ]:
application.search_profile(selectors=[{'people': ['id',
                                                  'first-name',
                                                  'last-name',
                                                  {'positions': {'company': 'id', 'name'}}
                                                 ]
                           }],
                           params={'keywords': 'apple microsoft'}
)
# Search URL is https://api.linkedin.com/v1/people-search:(people:(first-name,last-name))?keywords=apple%20microsoft

In [ ]:
url = 'https://api.linkedin.com/v1/people-search:(people:(first-name,last-name))?keywords=apple%20microsoft'
head = {'Authorization': 'access_token {}'.format(access_token)}
response = requests.get(url, headers=head)

response.content